### Visualizing synthetic data created using Van der Pol Oscillator dynamics



In [ ]:
# imports
import matplotlib
%matplotlib inline
# %matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

import copy
import h5py
import torch
import scipy
import scipy.ndimage
from einops import rearrange

# local imports
import sys
sys.path.append("..")
from code_pack.plotting import plot_two_d_vector_field_from_data, raster_to_events
from code_pack.generate_vdp_data import generate_van_der_pol, generate_noisy_van_der_pol

We consider an Euler discretization of a Van der Pol oscillator with noisy transitions:

$z_{t+1,1} = z_{t,1} + \tau_1^{-1} \Delta z_{t,2} + \sigma \epsilon$\
$z_{t+1,2} = z_{t,2} + \tau_2^{-1} \Delta(\mu (1-z_{t,1})^2 z_{t,2} - z_{t,1}) + \sigma \epsilon$

and Poisson observations:

$y_{n,t} \sim \text{Poisson}(y_{n,t} \mid \Delta \exp(C_n^\top z_t + b_n))$

In [ ]:
# loading data from ./data/vdp_noisy.h5
file_name = "data/poisson_obs.h5"

# dynamics parameters
data = h5py.File(file_name, 'r')
system_parameters = {}
system_parameters['mu'] = data['mu']
system_parameters['tau_1'] = data['tau_1']
system_parameters['tau_2'] = data['tau_2']
system_parameters['sigma'] = data['sigma']
system_parameters['scale'] = np.array(data['scale'])

### Visualizationing trajectories

In [ ]:
# plotting trajectories of the dataset
X = np.array(data['X'])
fig, ax = plt.subplots(1, 1, figsize=(5,5))
_ = ax.plot(X[0,:,0], X[0,:,1])
ax.scatter(X[0, 0, 0], X[0, 0, 1], marker='o', color='red', zorder=10, s=100, label='start')
ax.scatter(X[0, -1, 0], X[0, -1, 1], marker='x', color='red', zorder=10, s=100, label='end')

# system_parameters_copy = copy.deepcopy(system_parameters)
system_parameters['sigma'] = 0.0
dynamic_func = lambda inp : generate_noisy_van_der_pol(inp, np.array([0.0, 5e-3]), system_parameters)
axs_range = {'x_min':-1.5, 'x_max':1.5, 'y_min':-1.5, 'y_max':1.5}
plot_two_d_vector_field_from_data(dynamic_func, ax, axs_range)

ax.legend()
ax.set_title('sample trajectory (true state)');

### Effect of tuning function


In [ ]:
C_tilde = np.array(data['C_tilde'])
idx = np.lexsort((C_tilde[:,0], C_tilde[:,1]), axis=0) # sort the loading

In [ ]:
# showing the spike raster generated from noisy Vdp
fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharex=True, sharey=True)
events = raster_to_events(np.array(data['Y'])[0,:,:])
events_softplus = raster_to_events(np.array(data['Y_softplus'])[0,:,:])
events_axis_aligned = raster_to_events(np.array(data['Y_axis'])[0,:,idx].transpose())
axs[0].eventplot(events, linewidths=0.5, color='blue');
axs[1].eventplot(events_softplus, linewidths=0.5, color='blue');
axs[2].eventplot(events_axis_aligned, linewidths=0.5, color='blue');
axs[0].set_title(f'$\exp()$');
axs[1].set_title(f'softplus$()$');
axs[2].set_title(f'axis aligned');
axs[0].set_xlabel("Time");
axs[0].set_ylabel("Neuron");

### PCA
The first latent variable model that most people become familiar with is PCA.
Before performing PCA, we smooth the spikes with a Gaussian kernel

In [ ]:
# smoothing data with a gaussian kernel
data_stacked = rearrange(np.array(data['Y']), 'trial time neurons -> (trial neurons) time')
data_smooth = scipy.ndimage.gaussian_filter1d(input = data_stacked, sigma=50.0, axis=1)
data_smooth = rearrange(data_smooth, '(trial neurons) time -> (trial time) neurons', trial=250, neurons=250) # TODO: hardcoded

In [ ]:
data_centered = data_smooth - np.mean(data_smooth, axis=0)

In [ ]:
tidx = slice(500,1000)
fig, ax = plt.subplots(1, 1, figsize =(10, 5))
ax.plot(data_centered[tidx, 0:3]);
ax.set_xlabel("Time");

In [ ]:
# PCA using SVD
u, s, vh = np.linalg.svd(data_centered, full_matrices=False)
u.shape, s.shape

In [ ]:
norm_sv = s**2/np.sum(s**2)
top2sv = np.sum(norm_sv[:2])
print("Total observations explained by the first two principal components: {0:.3f}%".format(top2sv*100))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 3))

axs[0].plot(norm_sv * 100, 'o-')
axs[1].plot(norm_sv.cumsum() * 100, 'o-')
axs[1].set_ylim([0, 100])
axs[2].plot(20*np.log10(norm_sv), 'o-')

[(axs[k].grid(), axs[k].set_title(f'')) for k in range(3)]
axs[0].set_ylabel("Variance explained per PC ($\%$)"); 
axs[1].set_ylabel("Cumulative variance explained ($\%$)");
axs[2].set_ylabel("Variance explained (dB)"); 
axs[0].set_xlabel("PC (ordered)");
axs[1].set_xlabel("PC (ordered)");
axs[2].set_xlabel("PC (ordered)");
fig.suptitle("What's the dimensionality? Inspecting variance explained by each PC defined dim");

In [ ]:
# visualizing top two PCs
top2u = u[:, :2]
top2s = s[:2]
top2reconstruction = top2u * top2s
print(top2reconstruction.shape)
fig, ax = plt.subplots(1, 1, figsize =(10, 5))
ax.plot(top2reconstruction[tidx, 0], color='blue', label='PC 1')
ax.plot(top2reconstruction[tidx, 1], color='red', label='PC 2')
ax.legend()
ax.set_xlabel("Time")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize =(10, 5))
nTrial = 10
cmap = matplotlib.colormaps['gist_rainbow'].resampled(nTrial)
for trial in range(nTrial):
    tidx = slice(500*trial, 500*(trial+1))
    ax.plot(top2reconstruction[tidx, 0], top2reconstruction[tidx, 1], color=cmap(trial))
ax.grid()

In [ ]:
A, _, _, _ = np.linalg.lstsq(top2reconstruction, X.reshape((-1,2)), rcond=None)

In [ ]:
b = top2reconstruction @ A
b = b.reshape((250,500,2))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize =(10, 5))
trial=2
ax.set_xlabel("Time")
ax.plot(b[trial, :, :], 'b', label="Reconstruction")
ax.plot(X[trial, :, :], 'r', label="Data")
ax.legend();

In [ ]:
fig, ax = plt.subplots(1, 1, figsize =(10, 5))
nTrial = 2
cmap = matplotlib.colormaps['gist_rainbow'].resampled(nTrial)
for trial in range(nTrial):
    ax.plot(b[trial, :, 0], b[trial, :, 1], color=cmap(trial))
    ax.plot(X[trial, :, 0], X[trial, :, 1], '--', color=cmap(trial))
ax.grid()